In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg

experiment:
  torch_compile: False
  torch_mixed_precision: True 

data:
  iters_per_epoch: '?' 
  train_splits: cal
  val_splits: val 

dataloader:
  batch_size: '?'
  num_workers: 1
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  lr: '?'

train:
  epochs: '?'
  eval_freq: '?'
  base_pretrained_dir: '?'
  base_checkpoint: 'max-val-dice_score'
  base_pt_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

In [ ]:
%%yaml log_cfg

log:
  checkpoint_freq: 50
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True
    ece_loss:
      _fn: ese.metrics.image_ece_loss
      num_prob_bins: 15
      from_logits: True
    area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: True
      square_diff: False

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - ece_loss 
          - area_estimation_error
        phase: val

In [ ]:
%%yaml experiment_cfg 

# name: "ISLES_3D_TS"
name: "ISLES_3D_LTS"

train: 
    base_pretrained_dir: 
        - "/storage/vbutoi/scratch/ESE/training/09_08_24_ISLES_3D_CrossEntropy"
        # - "/storage/vbutoi/scratch/ESE/training/09_08_24_ISLES_3D_Dice"

data:
    _class: ese.datasets.ISLES

model:
    dims: 3
    class_name: 
      # - TempScaling
      - LTS

dataloader:
  batch_size: 1 

optim:
  lr: 1.0e-5 


loss_func:
    _class: ese.losses.AreaEstimationError

In [ ]:
from ese.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([log_cfg, callbacks_cfg])

updated_base_cfg, cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs={}
)

In [ ]:
len(cal_cfgs)

## Running Jobs

In [ ]:
from ese.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

In [ ]:
####### Run individual jobs
run_ese_exp(
    config=cal_cfgs[0], 
    experiment_class=PostHocExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='0',
    # gpu='1',
)

In [ ]:
# #### Run Batch Jobs
# submit_ese_exps(
#     group="calibration",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=cal_cfgs,
#     experiment_class=PostHocExperiment,
#     track_wandb=True,
#     # available_gpus=['0', '1', '2', '3']
#     available_gpus=['0', '2', '4', '6'],
#     # available_gpus=['1', '3', '5', '7'],
#     # available_gpus=['4', '5', '6', '7']
# )